In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import urllib,time
import tushare as ts

# 1. 极端值涨跌幅预测因子

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Aug  6 14:31:49 2019

@author: hurenjie
"""

# 极端值因子：当大盘(或者其他股票)日涨幅/跌幅绝对值超过2%，下一天上涨或者下跌的概率/历史上的次数有多少
#%%

token='2f31c3932ead9fcc3830879132cc3ec8df3566550f711889d4a30f67'
pro = ts.pro_api(token)
df = ts.get_hist_data('sh').sort_index()

#%%
def extreme_value(df, threshold): # threshold 在此应该定义为一个百分数,代表我们认为的涨跌幅度阈值
    df['close_pct_ch'] = df['close'].pct_change().shift(-1)
    df['close_diff'] = df['close'].diff().shift(-2)
    df_up = df[df['close_diff'] > 0]
    df_down = df[df['close_diff'] < 0]
    
    up = df_up[df_up['close_pct_ch'] > threshold]
    down = df_down[df_down['close_pct_ch'] < -threshold]
    
    prob_up = np.shape(up)[0]/np.shape(df_up)[0]
    prob_down = np.shape(down)[0]/np.shape(df_down)[0]
        
    if df['close_diff'].iloc[-3] > 0:
        result = prob_up # 当天价格上涨，利用极端值因子估计法估计出第二天继续上涨的概率
        print('当天价格上涨,该函数计算的是第二天价格继续上涨的概率')
    else:
        result = prob_down # 当天价格下跌，利用极端值因子估计法估计出第二天继续下跌的概率
        print('当天价格下跌,该函数计算的是第二天价格继续下跌的概率')
    df.drop(columns = ['close_pct_ch', 'close_diff'], inplace = True)
    return result

In [3]:
extreme_value(df, 0.0145)

当天价格下跌,该函数计算的是第二天价格继续下跌的概率


0.06462585034013606

In [4]:
# threshold = 0.0156
# df['close_pct_ch'] = df['close'].pct_change().shift(-1)
# df['close_diff'] = df['close'].diff().shift(-2)
# df_up = df[df['close_diff'] > 0]
# df_down = df[df['close_diff'] < 0]

# up = df_up[df_up['close_pct_ch'] > threshold]
# down = df_down[df_down['close_pct_ch'] < -threshold]

# prob_up = np.shape(up)[0]/np.shape(df_up)[0]
# prob_down = np.shape(down)[0]/np.shape(df_down)[0]

# if df['close_diff'].iloc[-3] > 0:
#     result = prob_up # 当天价格上涨，利用极端值因子估计法估计出第二天继续上涨的概率
#     print('当天价格上涨,该函数计算的是第二天价格继续上涨的概率')
# else:
#     result = prob_down # 当天价格下跌，利用极端值因子估计法估计出第二天继续下跌的概率
#     print('当天价格下跌,该函数计算的事第二天价格继续下跌的概率')
# # df.drop(columns = ['close_pct_ch', 'close_diff'], inplace = True)

# 2. 简单连续涨跌预测因子

In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Aug  6 16:15:22 2019

@author: hurenjie
"""
# 简单连续涨跌因子：连续涨/跌N（3、4、5、6）天之后，下一天涨/跌的概率
#%% 
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
# import tushare as ts

#token='2f31c3932ead9fcc3830879132cc3ec8df3566550f711889d4a30f67'
#pro = ts.pro_api(token)
#df = ts.get_hist_data('sh').sort_index()

#%%
def simple_predict_up_or_down(df, N): # N代表我们认为的连续多少天持续的涨或者跌
    df.reset_index(inplace = True)
    df['close_diff'] = df['close'].diff()
    df['signal'] = 0
    for i in df.index:
        if df['close_diff'][i] < 0:
            df['signal'][i] = -1
        elif df['close_diff'][i] > 0:
            df['signal'][i] = 1
    
    df['signal_sum'] = df['signal'].rolling(N).sum()
    df_up_Ndays = df[df['signal_sum'] == N]
    next_day_is_up = 0
    
    df_down_Ndays = df[df['signal_sum'] == -N]
    next_day_is_down = 0
    
    for i in df_up_Ndays.index:
        if i!=df.index[-1]:
            if df['close_diff'][i+1] > 0:
                next_day_is_up += 1
    
    for i in df_down_Ndays.index:
        if i!=df.index[-1]:
            if df['close_diff'][i+1] < 0:
                next_day_is_down += 1
    df.drop(columns = ['close_diff', 'signal', 'signal_sum'], inplace = True)
    df.set_index('date', inplace = True)
    
    prob_up = next_day_is_up/np.shape(df_up_Ndays)[0]
    prob_down = next_day_is_down/np.shape(df_down_Ndays)[0]
    
    return {'连续'+ str(N) +'天上涨后第二天上涨的概率': prob_up, '连续'+ str(N) +'天下跌后第二天下跌的概率':prob_down}

In [5]:
simple_predict_up_or_down(df, 4)

NameError: name 'simple_predict_up_or_down' is not defined

# 3. 乖离率反弹因子

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Aug  7 10:26:08 2019

@author: hurenjie
"""
# 乖离率绝对值太大时，或者大到什么幅度时，大概率回归？
#%%
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import tushare as ts
import talib

token='2f31c3932ead9fcc3830879132cc3ec8df3566550f711889d4a30f67'
pro = ts.pro_api(token)
df = ts.get_hist_data('sh').sort_index()
df.index = pd.to_datetime(df.index)
#%%
def bias_next_day_prediction(df, zscorethreshold, bias_time_window=30, zscore_time_window=30, return_date = False):
    df['close_diff'] = df['close'].diff().shift(-1)
    
#    compute bias
    ave = df['close'].rolling(bias_time_window).mean()
    bias = (df['close']-ave)/ave
    
#    compute zscore
    df['bias'] = bias
    df['zscore'] = (df['bias'] - df['bias'].rolling(zscore_time_window).mean())/df['bias'].rolling(zscore_time_window).apply(np.std)
    
    larger_than_positive_threshold = df[df['zscore'] >= zscorethreshold]
    smaller_than_negative_threshold = df[df['zscore'] <= zscorethreshold]
    
    larger_than_positive_threshold_nextday_down = larger_than_positive_threshold[larger_than_positive_threshold['close_diff']<0]
    smaller_than_negative_threshold_nextday_up = smaller_than_negative_threshold[smaller_than_negative_threshold['close_diff']>0]
    
    if (df.index[-1] not in larger_than_positive_threshold.index) & (df.index[-1] not in smaller_than_negative_threshold.index):
        print(str(df.index[-1].date()) + '并没有发生z-score超出阈值'+str(zscorethreshold)+'的现象')
    elif (df.index[-1] in larger_than_positive_threshold.index):
        print(str(df.index[-1].date()) + '向上超出了z-score设定的阈值'+str(zscorethreshold))
    elif (df.index[-1] in smaller_than_negative_threshold.index):
        print(str(df.index[-1].date()) + '向下超出了z-score设定的阈值'+str(zscorethreshold))
    
    try:
        larger_than_positive_threshold_nextday_down_prob = larger_than_positive_threshold_nextday_down.shape[0]/larger_than_positive_threshold.shape[0]
        smaller_than_negative_threshold_nextday_up_prob = smaller_than_negative_threshold_nextday_up.shape[0]/smaller_than_negative_threshold.shape[0]
#        df.drop(columns = ['close_diff', 'bias', 'zscore'], inplace =  True)
    except:
        print('阈值选取不合适，请选取小一点的阈值 zscorethreshold')
        df.drop(columns = ['close_diff', 'bias', 'zscore'], inplace =  True)
    else:
        df.drop(columns = ['close_diff', 'bias', 'zscore'], inplace =  True)
        if return_date == False:
            return {'输入的数据中乖离率zscore大于'+str(zscorethreshold)+'时第二天下跌的概率':larger_than_positive_threshold_nextday_down_prob,
                    '输入的数据中乖离率zscore小于'+str(zscorethreshold)+'时第二天上涨的概率':smaller_than_negative_threshold_nextday_up_prob}
        elif return_date == True :
            return {'输入的数据中乖离率zscore大于'+str(zscorethreshold)+'时第二天下跌的概率':larger_than_positive_threshold_nextday_down_prob,
                    '输入的数据中乖离率zscore小于'+str(zscorethreshold)+'时第二天上涨的概率':smaller_than_negative_threshold_nextday_up_prob,
                    '向上超出zscore阈值'+str(zscorethreshold)+'的时间点':list(larger_than_positive_threshold.index),
                    '向下超出zscore阈值'+str(zscorethreshold)+'的时间点':list(smaller_than_negative_threshold.index)}
    

#%%
bias_next_day_prediction(df, zscorethreshold=3, return_date = False)

ModuleNotFoundError: No module named 'talib'

# 网易爬数据

In [2]:
def get_page(url):  #获取页面数据
    req=urllib.request.Request(url,headers={
        'Connection': 'Keep-Alive',
        'Accept': 'text/html, application/xhtml+xml, */*',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko'
    })
    opener=urllib.request.urlopen(req)
    page=opener.read()
    return page

def get_index_history_byNetease(index_temp):
    """
    :param index_temp: for example, 'sh000001' 上证指数
    :return:
    """
    index_type=index_temp[0:2]
    index_id=index_temp[2:]
    if index_type=='sh':
        index_id='0'+index_id
    if index_type=="sz":
        index_id='1'+index_id
    url='http://quotes.money.163.com/service/chddata.html?code=%s&start=19900101&end=%s&fields=TCLOSE;HIGH;LOW;TOPEN;LCLOSE;CHG;PCHG;VOTURNOVER;VATURNOVER'%(index_id,time.strftime("%Y%m%d"))

    page=get_page(url).decode('gb2312') #该段获取原始数据
    page=page.split('\r\n')
    col_info=page[0].split(',')   #各列的含义
    index_data=page[1:]     #真正的数据

    #为了与现有的数据库对应，这里我还修改了列名，大家不改也没关系
    col_info[col_info.index('日期')]='交易日期'   #该段更改列名称
    col_info[col_info.index('股票代码')]='指数代码'
    col_info[col_info.index('名称')]='指数名称'
    col_info[col_info.index('成交金额')]='成交额'

    index_data=[x.replace("'",'') for x in index_data]  #去掉指数编号前的“'”
    index_data=[x.split(',') for x in index_data]

    index_data=index_data[0:index_data.__len__()-1]   #最后一行为空，需要去掉
    pos1=col_info.index('涨跌幅')
    pos2=col_info.index('涨跌额')
    posclose=col_info.index('收盘价')
    index_data[index_data.__len__()-1][pos1]=0      #最下面行涨跌额和涨跌幅为None改为0
    index_data[index_data.__len__()-1][pos2]=0
    for i in range(0,index_data.__len__()-1):       #这两列中有些值莫名其妙为None 现在补全
        if index_data[i][pos2]=='None':
            index_data[i][pos2]=float(index_data[i][posclose])-float(index_data[i+1][posclose])
        if index_data[i][pos1]=='None':
            index_data[i][pos1]=(float(index_data[i][posclose])-float(index_data[i+1][posclose]))/float(index_data[i+1][posclose])

    # print(col_info)
    return [index_data,col_info]
# --------------------- 
# 版权声明：本文为CSDN博主「multiangle」的原创文章，遵循CC 4.0 by-sa版权协议，转载请附上原文出处链接及本声明。
# 原文链接：https://blog.csdn.net/u014595019/article/details/48445223

In [3]:
sh = get_index_history_byNetease("sh000001") 
df_ne = pd.DataFrame()
df_ne['columns_to_split'] = pd.DataFrame(sh).T[0]
col_list = list(pd.DataFrame(sh).T[1])[:12]
df_ne['columns_to_split'] = df_ne['columns_to_split'].apply(lambda x: str(x))
df_ne['columns_to_split'] = df_ne['columns_to_split'].apply(lambda x: x.replace("[", ""))
df_ne['columns_to_split'] = df_ne['columns_to_split'].apply(lambda x: x.replace("]", ""))

df_ne = df_ne['columns_to_split'].str.split(",", 12, expand = True)
df_ne.columns = col_list
df_ne.sort_values("交易日期", inplace = True)
df_ne.reset_index(inplace = True)
del df_ne['index']
df_ne.tail()

,交易日期,指数代码,指数名称,收盘价,最高价,最低价,开盘价,前收盘,涨跌额,涨跌幅,成交量,成交额
7046,'2019-10-17','000001','上证指数','2977.3342','2986.7191','2969.568','2979.9929','2978.7124','-1.3782','-0.0463','122527614','1.36051252104e+11'
7047,'2019-10-18','000001','上证指数','2938.1413','2987.2035','2933.2424','2982.3417','2977.3342','-39.1929','-1.3164','149990678','1.62018272294e+11'
7048,'2019-10-21','000001','上证指数','2939.6179','2940.3246','2917.6884','2933.8969','2938.1413','1.4766','0.0503','132475107','1.4568588122e+11'
7049,'2019-10-22','000001','上证指数','2954.3799','2954.6415','2934.3873','2945.6001','2939.6179','14.762','0.5022','117482048','1.30381741175e+11'
7050,'2019-10-23','000001','上证指数','2941.6188','2957.2995','2934.6274','2952.9693','2954.3799','-12.7611','-0.4319','123812881','1.39653650476e+11'


In [4]:
df_ne = df_ne[['交易日期', '开盘价', '最高价', '最低价', '收盘价', '成交量']]
df_ne = df_ne.iloc[1:, :]

In [6]:
df_ne.head()

,交易日期,开盘价,最高价,最低价,收盘价,成交量
1,'1990-12-20','104.3','104.39','99.98','104.39','197'
2,'1990-12-21','109.07','109.13','103.73','109.13','28'
3,'1990-12-24','113.57','114.55','109.13','114.55','32'
4,'1990-12-25','120.09','120.25','114.55','120.25','15'
5,'1990-12-26','125.27','125.27','120.25','125.27','100'


In [7]:
df_ne['交易日期'] = df_ne['交易日期'].apply(lambda x: x[1:-1])
df_ne['收盘价'] = df_ne['收盘价'].apply(lambda x: x[2:-1])
df_ne['收盘价'] = df_ne['收盘价'] = df_ne['收盘价'].apply(lambda x: float(x))
df_ne['最高价'] = df_ne['最高价'].apply(lambda x: x[2:-1])
df_ne['最高价'] = df_ne['最高价'] = df_ne['最高价'].apply(lambda x: float(x))
df_ne['最低价'] = df_ne['最低价'].apply(lambda x: x[2:-1])
df_ne['最低价'] = df_ne['最低价'] = df_ne['最低价'].apply(lambda x: float(x))
df_ne['开盘价'] = df_ne['开盘价'].apply(lambda x: x[2:-1])
df_ne['开盘价'] = df_ne['开盘价'] = df_ne['开盘价'].apply(lambda x: float(x))
# df_ne['前收盘'] = df_ne['前收盘'].apply(lambda x: x[2:-1])
# df_ne['前收盘'] = df_ne['前收盘'] = df_ne['前收盘'].apply(lambda x: float(x))
# df_ne['涨跌额'] = df_ne['涨跌额'].apply(lambda x: x[2:-1])
# df_ne['涨跌额'] = df_ne['涨跌额'] = df_ne['涨跌额'].apply(lambda x: float(x))
# df_ne['涨跌幅'] = df_ne['涨跌幅'].apply(lambda x: x[2:-1])
# df_ne['涨跌幅'] = df_ne['涨跌幅'] = df_ne['涨跌幅'].apply(lambda x: float(x))
df_ne['成交量'] = df_ne['成交量'].apply(lambda x: x[2:-1])
df_ne['成交量'] = df_ne['成交量'] = df_ne['成交量'].apply(lambda x: float(x))
# df_ne['成交额'] = df_ne['成交额'].apply(lambda x: x[2:-1])
# df_ne['成交额'] = df_ne['成交额'] = df_ne['成交额'].apply(lambda x: float(x))

In [9]:
df_ne.tail()

,交易日期,开盘价,最高价,最低价,收盘价,成交量
7046,2019-10-17,2979.9929,2986.7191,2969.5680,2977.3342,122527614.0
7047,2019-10-18,2982.3417,2987.2035,2933.2424,2938.1413,149990678.0
7048,2019-10-21,2933.8969,2940.3246,2917.6884,2939.6179,132475107.0
7049,2019-10-22,2945.6001,2954.6415,2934.3873,2954.3799,117482048.0
7050,2019-10-23,2952.9693,2957.2995,2934.6274,2941.6188,123812881.0


In [10]:
df_ne.to_csv("20191023上证指数数据.csv", index = False)

In [61]:
df_ne.columns = ['date', 'open', 'high', 'low', 'close', 'volume']

In [64]:
df_ne

,date,open,high,low,close,volume
1,1990-12-20,104.3000,104.3900,99.9800,104.3900,197.0
2,1990-12-21,109.0700,109.1300,103.7300,109.1300,28.0
3,1990-12-24,113.5700,114.5500,109.1300,114.5500,32.0
4,1990-12-25,120.0900,120.2500,114.5500,120.2500,15.0
5,1990-12-26,125.2700,125.2700,120.2500,125.2700,100.0
6,1990-12-27,125.2700,125.2800,125.2700,125.2800,66.0
7,1990-12-28,126.3900,126.4500,125.2800,126.4500,108.0
8,1990-12-31,126.5600,127.6100,126.4800,127.6100,78.0
9,1991-01-02,127.6100,128.8400,127.6100,128.8400,91.0
10,1991-01-03,128.8400,130.1400,128.8400,130.1400,141.0


In [6]:
def extreme_value(df, threshold): # threshold 在此应该定义为一个百分数,代表我们认为的涨跌幅度阈值
    df['close_pct_ch'] = df['close'].pct_change().shift(-1)
    df['close_diff'] = df['close'].diff().shift(-2)
    df_up = df[df['close_diff'] > 0]
    df_down = df[df['close_diff'] < 0]
    
    up = df_up[df_up['close_pct_ch'] > threshold]
    down = df_down[df_down['close_pct_ch'] < -threshold]
    
    prob_up = np.shape(up)[0]/np.shape(df_up)[0]
    prob_down = np.shape(down)[0]/np.shape(df_down)[0]
        
    if df['close_diff'].iloc[-3] > 0:
        result = prob_up # 当天价格上涨，利用极端值因子估计法估计出第二天继续上涨的概率
        print('当天价格上涨,该函数计算的是第二天价格继续上涨的概率')
    else:
        result = prob_down # 当天价格下跌，利用极端值因子估计法估计出第二天继续下跌的概率
        print('当天价格下跌,该函数计算的是第二天价格继续下跌的概率')
    df.drop(columns = ['close_pct_ch', 'close_diff'], inplace = True)
    return result

In [7]:
df_ne.columns = ['交易日期', '指数代码', '指数名称', 'close', '最高价', '最低价', '开盘价', '前收盘', '涨跌额', '涨跌幅',
       '成交量', '成交额']

In [8]:
df_ne

,交易日期,指数代码,指数名称,close,最高价,最低价,开盘价,前收盘,涨跌额,涨跌幅,成交量,成交额
7005,'1990-12-19','000001','上证指数','99.98','99.98','95.79','96.05','None',0,0,'1260','494000.0'
7004,'1990-12-20','000001','上证指数','104.39','104.39','99.98','104.3','99.98','4.41','4.4109','197','84000.0'
7003,'1990-12-21','000001','上证指数','109.13','109.13','103.73','109.07','104.39','4.74','4.5407','28','16000.0'
7002,'1990-12-24','000001','上证指数','114.55','114.55','109.13','113.57','109.13','5.42','4.9666','32','31000.0'
7001,'1990-12-25','000001','上证指数','120.25','120.25','114.55','120.09','114.55','5.7','4.976','15','6000.0'
7000,'1990-12-26','000001','上证指数','125.27','125.27','120.25','125.27','120.25','5.02','4.1746','100','53000.0'
6999,'1990-12-27','000001','上证指数','125.28','125.28','125.27','125.27','125.27','0.01','0.008','66','104000.0'
6998,'1990-12-28','000001','上证指数','126.45','126.45','125.28','126.39','125.28','1.17','0.9339','108','88000.0'
6997,'1990-12-31','000001','上证指数','127.61','127.61','126.48','126.56','126.45','1.16','0.9174','78','60000.0'
6996,'1991-01-02','000001','上证指数','128.84','128.84','127.61','127.61','127.61','1.23','0.9639','91','59000.0'


In [9]:
df_ne['close'] = df_ne['close'].apply(lambda x: x[2:-1])
df_ne['close'] = df_ne['close'].apply(lambda x: float(x))

In [10]:
extreme_value(df_ne, -0.0063)

当天价格下跌,该函数计算的是第二天价格继续下跌的概率


0.7204950196196801